##  Decision Trees for Regression

We can build decision trees for regression just like we built regression trees for classification.  The primary difference is the response is numeric for a regression tree.  This means that trees are split on a different metric than *gini impurity* which we used for classification trees.  Below we will use *mean squared error* which is the default to make our splits/branches but there are other choices available such as *mean absolute error*.

One thing to emphasis about decision trees is that the features much be numeric.  That means if we have categorical features/predictors/covariates that we will have to create indicators or one-hot encoded values for those features.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from matplotlib import colors
import seaborn as sns

import scipy.stats as stats


from sklearn import tree

from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance

In [ ]:
# read in the data to dataframe called ames
ames = pd.read_csv("https://webpages.charlotte.edu/mschuck1/classes/DTSC2301/Data/Ames_house_prices.csv", na_values=['?'])
# replace the ? in the data with NaN for missing values
ames.replace([' ?'],np.nan)
# show information about the dataframe
ames.info()

In [ ]:
y=ames['SalePrice']
#ames=ames.drop('SalePrice', axis=1)
# we have to have numeric predictors for a decision tree
X=ames[['LotFrontage','LotArea','ScreenPorch','MoSold','YearBuilt','YearRemodAdd', 'BsmtFinSF1','OpenPorchSF','PoolArea','1stFlrSF','2ndFlrSF','GrLivArea']]
print(X.head())

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a regression tree model
dtree = DecisionTreeRegressor(random_state=0)

# Train the model
dtree.fit(X_train, y_train)

# Make predictions
predictions = dtree.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"Mean Squared Error: {rmse}")

In [ ]:
# create a plot of the decision tree
fig = plt.figure(figsize=(25,20))
tree.plot_tree(dtree,
                   feature_names=['LotFrontage','LotArea','ScreenPorch','MoSold','YearBuilt','YearRemodAdd', 'BsmtFinSF1','OpenPorchSF','PoolArea','1stFlrSF','2ndFlrSF','GrLivArea'],
                   filled=True)

Whoo-boy. That is a tree.  Perhaps we should prune that tree.

In [ ]:
# initiate a decision tree with a max depth of 4
dt_pre_pruned = DecisionTreeRegressor(max_depth=4, min_samples_split=5, min_samples_leaf=2)

# Train the model
dt_pre_pruned.fit(X_train, y_train)

Let's get the feature importance for this model.  We'll use permutation importance for this model on the test data.

Reminder that the permutation importance is the amount that the model performance changes if we permute the values in that feature while keeping other features intact.    

In [ ]:
# Permutation feature importance

result = permutation_importance(dt_pre_pruned, X_test, y_test, n_repeats=10, random_state=0, n_jobs=-1)
perm_imp_df = pd.DataFrame({'Feature': feature_names, 'Permutation Importance': result.importances_mean}).sort_values('Permutation Importance', ascending=False)
print(perm_imp_df)

The values we get here make a good bit of sense as to the value of a house from my perspective.  *GrLivArea* is the amount of above grade (above ground?) living space and
the age of a house, *YearBuilt*, certainly have impacts on prices for houses.  


In [ ]:
# Make predictions
predictions = dt_pre_pruned.predict(X_test)
 
# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"Mean Squared Error: {rmse}")

In [ ]:

fig = plt.figure(figsize=(25,20))
tree.plot_tree(dt_pre_pruned,
                   feature_names=['LotFrontage','LotArea','ScreenPorch','MoSold','YearBuilt','YearRemodAdd', 'BsmtFinSF1','OpenPorchSF','PoolArea','1stFlrSF','2ndFlrSF','GrLivArea'],
                   filled=True)

Our out of sample performance is not as good but let's try something smaller.

In [ ]:
dt_pre_pruned2 = DecisionTreeRegressor(max_depth=2, min_samples_split=5, min_samples_leaf=2)

# Train the model
dt_pre_pruned2.fit(X_train, y_train)
# Make predictions
predictions = dt_pre_pruned2.predict(X_test)
 

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"Mean Squared Error: {rmse}")

In [ ]:
fig = plt.figure(figsize=(25,20))
tree.plot_tree(dt_pre_pruned2,
                   feature_names=['LotFrontage','LotArea','ScreenPorch','MoSold','YearBuilt','YearRemodAdd', 'BsmtFinSF1','OpenPorchSF','PoolArea','1stFlrSF','2ndFlrSF','GrLivArea'],
                   filled=True)
feature_names=['LotFrontage','LotArea','ScreenPorch','MoSold','YearBuilt','YearRemodAdd', 'BsmtFinSF1','OpenPorchSF','PoolArea','1stFlrSF','2ndFlrSF','GrLivArea']

So with that depth we did not get a better tree but we did get one we can visualize.

Let's look at the cross-validation rather than a single test set.  We'll use 8-fold CV and we'll look at trees with depth 2 and depth 4.

In [ ]:
dt_pre_pruned3 = DecisionTreeRegressor(max_depth=2, min_samples_split=5, min_samples_leaf=2)
cv_scores_tree3 = cross_val_score(dt_pre_pruned3, X, y, cv=8, scoring='neg_root_mean_squared_error')  # 8-fold cross-validation
print(f"RBF Kernel SVM cross-validation accuracy: {cv_scores_tree3.mean() :.2f}")

In [ ]:
dt_pre_pruned3 = DecisionTreeRegressor(max_depth=4, min_samples_split=5, min_samples_leaf=2)
cv_scores_tree3 = cross_val_score(dt_pre_pruned3, X, y, cv=8, scoring='neg_root_mean_squared_error')  # 8-fold cross-validation
print(cv_scores_tree3)
print(f"RBF Kernel SVM cross-validation accuracy: {cv_scores_tree3.mean() :.2f}")

So from above we can see that the larger tree with *max_depth=4* performed better on cross-validation.  Below is code to determine the
best *max_depth* via cross-validation. 

In [ ]:
sizes = 12

# create an empty array to store the calculated mean RMSEs
aver_rmse = []
depth=list(range(1,13,1))
# loop through the different pruning sizes
for i in range(sizes):
  # create a pruned tree of size i+1
  dt_pruned = DecisionTreeRegressor(max_depth=i+1, min_samples_split=5, min_samples_leaf=2)
  # calculate the negative RMSE, why python insists on the negative here I do not know
  # ...
  # well...
  # actually I have a guess.  I'm guessing that the algorithm is set to maximize a quantity
  # such as r^2.  And maximizing -RMSE is the same as minimizing RMSE
  cv_scores_tree = cross_val_score(dt_pruned, X, y, cv=8, scoring='neg_root_mean_squared_error')
  # get the average from the 8-fold cross validations and multiply by -1 to get back to the right scale. 
  avg = -1*cv_scores_tree.mean()
  # add avg to the list of other bootstrapped means
  aver_rmse.append(avg)


In [ ]:
# look at the values
print(aver_rmse)

Let's look at a plot of these cross validated RMSE's to get a better sense of these.

In [ ]:
# make a scatterplot for depth vs average RMSE
ax=sns.scatterplot(x=depth, y=aver_rmse)
# "ax" is the conventional name.
ax.set_title('Plot of CV RMSE by pruned depth')
ax.set_ylabel('CV RMSE')
ax.set_xlabel('prune depth')


It looks like the 'best' performing prune depth is 7 or 8 or 9.  I'd prefer the smallest model here so that seems to be *7*.  



### Forests from Trees

The next cells of code move from regression decision trees to regression random forests.

In [ ]:


# Train Random Forest Regressor with maximum depth of 3 
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=3, random_state=42)
# get the 8 fold cross validation score
scores = -1*cross_val_score(rf_regressor, X, y, cv=8, scoring='neg_root_mean_squared_error')
print(scores.mean())

So the above output was from making many trees with just a depth of 3.  Note that the average RMSE from our plot above with a prune depth of *3* was about 50000.  We got a good bit smaller RMSE by using a random forest of the same size.


In [ ]:
# Train Random Forest Regressor with maximum depth of 3 
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=4, random_state=42)
# get the 8 fold cross validation score
scores = -1*cross_val_score(rf_regressor, X, y, cv=8, scoring='neg_root_mean_squared_error')
print(scores.mean())

From that result, we can see that a random forest with a depth of *4* continues our improvement over having a depth of *3*.  In the Tasks below, you'll create a loop for
determining the 'best' value based upon cross-validation.

### Tasks

1. Write code to determine the best *max_depth* for a regression random forest for these data.  Make a plot like we did above to visualize the output.

2. Determine the permutation importance for the best regression random forest based upon your results from above.

3. Choose a subset of features/predictors from your results in Task 2.  Rerun the code for Tasks 1 and 2 with these new predictors.  Does your CV RMSE change?  If so, how?